In [3]:
import geopandas as gpd
import pandas as pd
import os
import fiona

In [4]:
carpeta_principal = r"C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas"
archivos_a_excluir = ["CCNN_PTOSINTERES.shp"]

In [5]:
gdf_list = []
for raiz, _, archivos in os.walk(carpeta_principal):
    for archivo in archivos:
        ruta_completa = os.path.join(raiz, archivo)

        # Ignorar archivos específicos
        if archivo in archivos_a_excluir:
            print(f"Ignorando archivo: {archivo}")
            continue  

        # Leer archivos SHP
        if archivo.endswith(".shp"):
            try:
                gdf = gpd.read_file(ruta_completa)
                
                # Convertir CRS si es diferente
                if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
                    gdf = gdf.to_crs("EPSG:4326")
                
                gdf["source"] = ruta_completa
                gdf_list.append(gdf)
            except Exception as e:
                print(f"Error al leer {ruta_completa}: {e}")

        # Leer archivos KML
        elif archivo.endswith(".kml"):
            try:
                # Obtener la lista de capas en el archivo KML
                layers = fiona.listlayers(ruta_completa)
                print(layers)
                
                # Intentar encontrar la capa que más se parece al nombre del archivo
                expected_layer = archivo[:-4]  # Nombre esperado
                matched_layer = None
                
                for layer in layers:
                    if expected_layer in layer:
                        matched_layer = layer
                        break
                
                if matched_layer is None:
                    print(f"No se encontró una capa coincidente en {ruta_completa}, usando la primera disponible.")
                    matched_layer = layers[0]  # Usar la primera capa si no hay coincidencias

                # Leer el archivo KML con la capa correcta
                gdf = gpd.read_file(ruta_completa, driver='KML', layer=matched_layer)

                # Convertir CRS si es diferente
                if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
                    gdf = gdf.to_crs("EPSG:4326")

                gdf["source"] = ruta_completa
                gdf_list.append(gdf)

            except Exception as e:
                print(f"Error al leer {ruta_completa}: {e}")


Ignorando archivo: CCNN_PTOSINTERES.shp
['1-100-SA23-SA_Monte_Perdido']
['1-101-SA4-SA_Faja_Pardina']
['1-102-SP12-SP_Las_Cutas']
['1-103-SP11-SP_La_Escala_y_Cobatar']
['1-104-SA14-SA_Faja_Cazcarra']
['1-105-SP13-SP_Torla_a_Fanlo_por_los_miradores_del_Molar_y_el_barranco_Borrue']
['1-106-SA27-SA_Marbore', 'Ramales']
['1-107-SP14-SP_Valle_Pineta_Solana']
['1-108-SA7-SA_Bestue', 'Ramales']
['1-109-SA12-SA_Foratarruego']
['1-332-SA32-SA_Faja_de_las_Flores']
['1-333-SP17-SP_Puerto_Bujaruelo']
['1-397-SP7-SP12-Enlace_SP_Puerto_de_Fanlo_-_SP_Las_Cutas']
['1-56-SA9-SA_Buerba_a_Vio']
['1-57-SP10-SP_Miradores_de_Revilla']
['1-58-SA13-SA_Fuentes_del_Yaga']
['1-59-SA24-SA_Soaso_a_Goriz']
['1-60-SA29-SA_Puerto_de_Lera']
['1-61-SA20-SA_Turieto', 'Ramales']
['1-62-SP3-SP_Sestrales']
['1-63-SA18-SA_Carriata']
['1-65-SA22-SA_Cuello_Gordo', 'Ramales']
['1-66-SIA1-SIA_San_Urbez_al_Molino_de_Aso']
['1-67-SA2-SA_Arrablo']
['1-68-SP4-SP_Fanlo_a_Sercue_por_Nerin']
['1-69-SA21-SA_Faja_Pelay']
['1-70-SA10-SA_

In [6]:
# Unificar todos los GeoDataFrames manteniendo todas las columnas (rellenando valores faltantes con NaN)
gdf_unificado = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True, join='outer'))

# Crear una columna de ID con un rango de números
gdf_unificado['ID_Ruta'] = range(len(gdf_unificado))

# Ver las primeras filas para comprobar que todo está correcto
gdf_unificado.head()

,ID,NOM_CAMINO,TIPOLOGIA,NOM_ETAPA,LONGITUD,ETAPA,CCAA,PROVINCIA,HUSO,ID_SECTOR,...,longitud,fecha_edi,recorrido,provincia,comunidad,estado,firme,nombre_en,fecha_edit,ID_Ruta
0,778.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 1: La Esperanza - La Caldera,30.924081,1,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,779.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 2: La Caldera - El Portillo,14.171262,2,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,780.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 3: El Portillo - Parador de Las Cañadas ...,16.686617,3,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,781.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 4: Parador de Las Cañadas del Teide - Vi...,15.420307,4,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,782.0,Camino Natural de Anaga - Chasna,CN por etapas (5),Etapa 5: Vilaflor - Arona,18.017360,5,Canarias,Santa Cruz de Tenerife,28,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [7]:
# Definir la ruta de salida
ruta_salida = os.path.join(carpeta_principal, "archivo_unificado.shp")

# Guardar el GeoDataFrame en un archivo SHP
gdf_unificado.to_file(ruta_salida)

print(f"Archivo unificado guardado en: {ruta_salida}")

C:\Users\maria.b.calvo\AppData\Local\Temp\ipykernel_6716\10481967.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_unificado.to_file(ruta_salida)


Archivo unificado guardado en: C:\Users\maria.b.calvo\Documents\Personal\Hackathon GenAI\hackathon-repo\Hackathon-PNAV\Datos\Datos Rutas\archivo_unificado.shp


C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Description' to 'Descriptio'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'id' to 'id_1'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'longitud' to 'longitud_1'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'provincia' to 'provinci_

In [8]:
# Imprimir nombres de las columnas (atributos)
print("Columnas del SHP:", gdf_unificado.columns)

Columnas del SHP: Index(['ID', 'NOM_CAMINO', 'TIPOLOGIA', 'NOM_ETAPA', 'LONGITUD', 'ETAPA',
       'CCAA', 'PROVINCIA', 'HUSO', 'ID_SECTOR', 'ID_CAMINO', 'COD_CAMINO',
       'ID_ETAPA', 'LONG_3D_GE', 'NUM_CAMINO', 'NOM_TOTAL', 'XCENTROIDE',
       'YCENTROIDE', 'geometry', 'source', 'Name', 'Description', 'id',
       'nombre', 'longitud', 'fecha_edi', 'recorrido', 'provincia',
       'comunidad', 'estado', 'firme', 'nombre_en', 'fecha_edit', 'ID_Ruta'],
      dtype='object')


In [9]:
gdf_atributos = gdf_unificado.copy()

# Diccionario de columnas equivalentes
column_mapping = {
    "Nombre_Ruta": ["NOM_CAMINO", "Name", "nombre"],
    "Longitud": ["LONGITUD", "longitud"],
    "Provincia": ["PROVINCIA", "provincia"],
    "CC_AA": ["CCAA", "comunidad"]
}

# Crear nuevas columnas con los valores existentes
for new_col, old_cols in column_mapping.items():
    gdf_atributos[new_col] = gdf_atributos[old_cols].bfill(axis=1).iloc[:, 0]  # Rellenar con el primer valor no nulo

# Eliminar las columnas antiguas si ya no se necesitan
columns_to_drop = sum(column_mapping.values(), [])  # Convertir listas de valores en una sola lista
gdf_atributos = gdf_atributos.drop(columns=columns_to_drop, errors="ignore")

# Imprimir nombres de las columnas (atributos)
print("Columnas del SHP:", gdf_atributos.columns)

# Crear una columna del nombre completo para aquellas rutas separadas por etapas
gdf_atributos['Nombre_completo'] = gdf_atributos['Nombre_Ruta'] + ' ' + gdf_atributos['NOM_ETAPA'].fillna('')

nuevo_orden = ['ID_Ruta', 'Nombre_Ruta', 'NOM_ETAPA', 'Nombre_completo', 'Longitud', 'Provincia', 'CC_AA', 
       'ID', 'TIPOLOGIA', 'ETAPA', 'HUSO', 'ID_SECTOR',
       'ID_CAMINO', 'COD_CAMINO', 'ID_ETAPA', 'LONG_3D_GE', 'NUM_CAMINO',
       'NOM_TOTAL', 'XCENTROIDE', 'YCENTROIDE', 'geometry', 'source',
       'Description', 'id', 'fecha_edi', 'recorrido', 'estado', 'firme',
       'nombre_en', 'fecha_edit']
gdf_atributos = gdf_atributos[nuevo_orden]

# Imprimir nombres de las columnas (atributos)
print("Columnas del SHP:", gdf_atributos.columns)

Columnas del SHP: Index(['ID', 'TIPOLOGIA', 'NOM_ETAPA', 'ETAPA', 'HUSO', 'ID_SECTOR',
       'ID_CAMINO', 'COD_CAMINO', 'ID_ETAPA', 'LONG_3D_GE', 'NUM_CAMINO',
       'NOM_TOTAL', 'XCENTROIDE', 'YCENTROIDE', 'geometry', 'source',
       'Description', 'id', 'fecha_edi', 'recorrido', 'estado', 'firme',
       'nombre_en', 'fecha_edit', 'ID_Ruta', 'Nombre_Ruta', 'Longitud',
       'Provincia', 'CC_AA'],
      dtype='object')
Columnas del SHP: Index(['ID_Ruta', 'Nombre_Ruta', 'NOM_ETAPA', 'Nombre_completo', 'Longitud',
       'Provincia', 'CC_AA', 'ID', 'TIPOLOGIA', 'ETAPA', 'HUSO', 'ID_SECTOR',
       'ID_CAMINO', 'COD_CAMINO', 'ID_ETAPA', 'LONG_3D_GE', 'NUM_CAMINO',
       'NOM_TOTAL', 'XCENTROIDE', 'YCENTROIDE', 'geometry', 'source',
       'Description', 'id', 'fecha_edi', 'recorrido', 'estado', 'firme',
       'nombre_en', 'fecha_edit'],
      dtype='object')


In [10]:
# Definir la ruta de salida
ruta_salida = os.path.join(carpeta_principal, "archivo_atributos_unificados.shp")

# Guardar el GeoDataFrame en un archivo SHP
gdf_atributos.to_file(ruta_salida)

C:\Users\maria.b.calvo\AppData\Local\Temp\ipykernel_6716\918135260.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_atributos.to_file(ruta_salida)
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre_Ruta' to 'Nombre_Rut'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Nombre_completo' to 'Nombre_com'
  ogr_write(
C:\Users\maria.b.calvo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Description' to 'Descriptio'
  ogr_write(
C:\Users\

In [11]:
# Definir la ruta de salida
ruta_salida = os.path.join(carpeta_principal, "archivo_rutas.csv")

# La longitud se expresa en decimales con punto decimal para evitar problemas con el formato CSV
gdf_atributos.to_csv(ruta_salida, index=False, encoding="utf-8-sig", decimal=",")

print("Archivo CSV creado con éxito.")


Archivo CSV creado con éxito.
